# Behavior Cloning Project

### Import Libraries

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow and tf.keras
from tensorflow import keras
import tensorflow as tf

# Helper libraries
import numpy as np
import pandas as pd
import sklearn
import math
import cv2

from sklearn.model_selection import train_test_split

import random
import csv
import matplotlib.pyplot as plt

from progressbar import *

print('Tensorflow Version: {}'.format(tf.__version__))

Tensorflow Version: 2.0.0


### Import Data

In [2]:
def generator(samples, batch_size=32):
    while True:
        random.shuffle(samples)
        for offset in range(0, len(samples), batch_size):
            subsamples = samples[offset:offset+batch_size]
            images, measurements = zip(*[[cv2.imread('data/' + smpl[0]), float(smpl[3])] for smpl in subsamples])
            X_train, y_train = (np.array(images), np.array(measurements))
            X_train, y_train = sklearn.utils.shuffle(X_train, y_train)
            yield (X_train, y_train)

In [3]:
reader = csv.reader(open('data/driving_log.csv'))
train_lines, valid_lines = train_test_split([line for line in reader][1:], test_size=0.2)

# images, measurements = zip(*[[cv2.imread('data/' + line[0]), float(line[3])] for line in progressbar(lines)])
batch_size=32
train = generator(train_lines, batch_size=batch_size)
valid = generator(valid_lines, batch_size=batch_size)
# X_train, y_train = (np.array(images), np.array(measurements))

### Define a Model

In [4]:
from tensorflow.keras.layers import *

model = tf.keras.Sequential()

model.add(Lambda(lambda x: (x/255)-0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((50,20), (0,0))))

# model.add(Conv2D(32, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2, 2)))
# model.add(Dropout(0.1))
# model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2, 2)))
# model.add(Dropout(0.3))
# model.add(Conv2D(64, (3, 3), activation='relu'))

model.add(Flatten())

model.add(Dense(64, activation='relu'))

model.add(Dense(1))

model.build()

model.summary()

model.compile(optimizer='adam', loss='mse')

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda (Lambda)              (None, 160, 320, 3)       0         
_________________________________________________________________
cropping2d (Cropping2D)      (None, 90, 320, 3)        0         
_________________________________________________________________
flatten (Flatten)            (None, 86400)             0         
_________________________________________________________________
dense (Dense)                (None, 64)                5529664   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 5,529,729
Trainable params: 5,529,729
Non-trainable params: 0
_________________________________________________________________


In [5]:
history = model.fit_generator(train, 
            steps_per_epoch=math.ceil(len(train_lines)/batch_size), 
            validation_data=valid, 
            validation_steps=math.ceil(len(valid_lines)/batch_size), 
            epochs=5, verbose=1)

Epoch 1/5
201/201 [==============================] - 64s 318ms/step - loss: 3.8760 - val_loss: 0.1301
Epoch 2/5
201/201 [==============================] - 60s 298ms/step - loss: 0.1606 - val_loss: 0.0366
Epoch 3/5
201/201 [==============================] - 62s 309ms/step - loss: 0.0371 - val_loss: 0.0258
Epoch 4/5
201/201 [==============================] - 58s 288ms/step - loss: 0.0260 - val_loss: 0.0434
Epoch 5/5
201/201 [==============================] - 59s 294ms/step - loss: 0.0235 - val_loss: 0.0174


### Save Model

In [6]:
model.save('my_model.h5')